<a href="https://colab.research.google.com/github/jiruneko/IntuitionLLM_structure/blob/main/IntuitionLLM_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# モデルとトークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
)

# パイプラインの作成
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

In [ ]:
!pip install -U "transformers==4.40.0" "accelerate" "safetensors"

In [ ]:
!pip install transformers==4.38.2 accelerate --upgrade

In [ ]:
!pip install -U transformers accelerate torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "microsoft/Phi-3.5-mini-instruct"  # こっちのほうが新しくて安定

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,      # 途中で切れないように長めに
    do_sample=True,
    temperature=0.7
)

prompt = (
    "あなたは日本語で丁寧なビジネスメールを書くアシスタントです。\n"
    "ガーデニングで起きてしまった悲しい出来事について、"
    "サラさんに謝罪するメールを書いてください。何が起きたのか、"
    "どのような経緯で起きてしまったのかも、わかりやすく説明してください。"
)

output = generator(prompt)
print(output[0]["generated_text"])


In [ ]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

model_output = model.model(input_ids)

lm_head_output = model.lm_head(model_output[0])

In [ ]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

In [ ]:
model_output[0].shape

In [ ]:
lm_head_output.shape

In [ ]:
prompt = "悲惨なガーデニングの失敗についてサラに謝罪する長いメールを書いてください。どのようにそれが起こったのか説明してください。"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [ ]:
%%timeit -n 1
# テキストを生成
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

In [ ]:
%%timeit -n 1
# テキストを生成
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)